In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('performance_data.csv', encoding='ISO-8859-1')

In [5]:
prompt_type = df.groupby('Conversation ID')['prompt type'].unique()

In [6]:
types = []

for i in prompt_type:
    types.append(list(i))
    
nested_list = types

In [7]:
flattened_list = [item for sublist in nested_list for item in sublist if item is not None]

In [8]:
unique_questions= df.drop_duplicates(subset=['Conversation ID', 'prompt type'])

In [15]:
unique_questions = pd.read_csv('unique_questions.csv')

In [62]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px


counts = unique_questions.groupby('prompt type').agg(count=('prompt type', 'size'), sum_b=('helpful', 'sum')).reset_index()
counts.columns = ['prompt type', 'question count','number of times chatbot answeredcorrectly']

In [102]:
unique_questions['correct'] = ['correct' if i == 1 else 'false' for i in unique_questions['helpful']]

In [112]:
unique_questions.sort_values('correct', inplace = True)

In [113]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd

app = dash.Dash(__name__)

# Assuming 'counts' is your DataFrame and it has a column 'c' for the new aggregate data
# counts = pd.DataFrame(...)  # Your existing DataFrame

counts_sorted = counts.sort_values(by='question count', ascending=False)
counts_sorted['Category'] = ['Top 6 topics' if i < 6 else 'Other topics' for i in range(len(counts_sorted))]


# First bar plot for 'count' column
fig = go.Figure(go.Bar(
    y=counts_sorted['prompt type'],
    x=counts_sorted['question count'],
    orientation='h',
    marker=dict(color='blue'),
    name='Count'
))

# Adding another set of bars for the 'c' column, overlaid
fig.add_trace(go.Bar(
    y=counts_sorted['prompt type'],
    x=counts_sorted['number of times chatbot answeredcorrectly'],
    orientation='h',
    marker=dict(color='green'),  # Semi-transparent orange
    name='Number of correct responses'
))

# Set the layout to overlay mode
fig.update_layout(
    barmode='overlay',
    title='Common customer inquiry topics received by Mcombo GPT',
    height=700,
    yaxis=dict(autorange='reversed')
)


counts_sorted = counts.sort_values(by='question count', ascending=False)

# Add a rectangular shape to highlight the top six bars
if len(counts_sorted) >= 6:
    fig.add_shape(
        type="rect",
        x0=0,
        y0=-0.5,  # Adjust as needed for precise positioning
        x1=counts_sorted['question count'].max()+3,  # or a suitable maximum value for your x-axis
        y1=5.5,  # Halfway between the 6th and 7th bar
        fillcolor="lightgrey",  # Choose a color that suits your plot's theme
        opacity=0.8,  # Adjust opacity for visibility of bars
        layer="below",  # Ensure the shape is below the bars
        line_width=0,
    )
    
fig.update_xaxes(title_text='Number of Inquiries')
fig.update_yaxes(title_text='Customer Inquiry')

app.layout = html.Div([
    dcc.Graph(
        id='bar-plot',
        figure=fig
    ),
    html.Div(id='text-output')
])

@app.callback(
    Output('text-output', 'children'),
    Input('bar-plot', 'clickData')
)
# def update_output(clickData):
#     if clickData is None:
#         return "Click on a bar!"
#     else:
#         value_clicked = clickData['points'][0]['y']
#         filtered_df = unique_questions[unique_questions['prompt type'] == value_clicked]['prompt'].head(20)
#         inquiries_list = [html.Li(item) for item in filtered_df]

#         inquiries_list.insert(0, html.P("All Inquiries:", style={'fontWeight': 'bold'}))
#         return html.Ul(inquiries_list)

def update_output(clickData):
    if clickData is None:
        return "Click on a bar!"
    else:
        # Get the value that was clicked
        value_clicked = clickData['points'][0]['y']
        
        # Filter the DataFrame based on the clicked value
        filtered_df = unique_questions[unique_questions['prompt type'] == value_clicked]

        # Create a list of HTML elements for each row in the filtered DataFrame
        inquiries_list = []
#         for _, row in filtered_df.iterrows():
#             item = html.Li([
#                 html.P(f"Conversation ID: {row['Conversation ID']}"),
#                 html.P(f"Customer question: {row['prompt']}"),
#                 html.P(f"Chatbot Response: {row['response']}"),
#                 html.P(f"Answered correctly: {row['correct']}"),
#             ])
#             inquiries_list.append(item)
        for _, row in filtered_df.iterrows():
            item = html.Li([
                html.P([html.Span("Conversation ID: ", style={'fontWeight': 'bold'}), html.Span(str(row['Conversation ID']))]),
                html.P([html.Span("Customer question: ", style={'fontWeight': 'bold'}), html.Span(str(row['prompt']))]),
                html.P([html.Span("Chatbot Response: ", style={'fontWeight': 'bold'}), html.Span(str(row['response']))]),
                html.P([html.Span("Answered correctly: ", style={'fontWeight': 'bold'}), html.Span(str(row['correct']))]),
            ])
            inquiries_list.append(item)
        # If you want to limit the number of displayed inquiries, you can slice the inquiries_list here
        # inquiries_list = inquiries_list[:20] 

        inquiries_list.insert(0, html.P("All Inquiries:", style={'fontWeight': 'bold'}))
        return html.Ul(inquiries_list)
    
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)